# CSC271H1 Week 04 Tutorial: Data Preparation and Analysis

In this tutorial, you will continue to work with the same **Canadian rental housing costs** dataset as last week.

<div class="alert alert-block alert-danger">
 <b>Important</b>: the autotesting will be inspecting the variables you are asked to create and their values, so be sure to use the variable names specified.
</div>

## Task 0: Load the data

Download the  `Canadian_rental_costs.csv` file from Quercus, put it in the same folder as this Jupyter notebook, and run the code cell below. Make sure the code runs without error before moving on to the next tasks.

In [60]:
import pandas as pd

df = pd.read_csv('Canadian_rental_costs.csv')
df.head()

,REF_DATE,GEO,DGUID,Rental unit type,Estimates,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2024-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,Average asking rent,Dollars,81,units,0,v1675424962,1.2.1,1150.0,NaN,NaN,NaN,0
1,2024-04,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,Average asking rent,Dollars,81,units,0,v1675424962,1.2.1,1050.0,E,NaN,NaN,0
2,2024-07,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,Average asking rent,Dollars,81,units,0,v1675424962,1.2.1,1200.0,NaN,NaN,NaN,0
3,2024-10,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,Average asking rent,Dollars,81,units,0,v1675424962,1.2.1,1230.0,NaN,NaN,NaN,0
4,2025-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,Average asking rent,Dollars,81,units,0,v1675424962,1.2.1,1210.0,NaN,NaN,NaN,0


## Task 1: Clean the data

### A: Remove Unnecessary Columns

In this step, you're going to remove columns that are not useful for our analysis.

There is a DataFrame method named `nunique` that returns how many unique values exist in each column. Let's run it now.

In [61]:
df.nunique()

REF_DATE              5
GEO                  42
DGUID                42
Rental unit type      3
Estimates             1
UOM                   1
UOM_ID                1
SCALAR_FACTOR         1
SCALAR_ID             1
VECTOR              126
COORDINATE          126
VALUE               191
STATUS                2
SYMBOL                0
TERMINATED            0
DECIMALS              1
dtype: int64

Looking at the output above, there are 6 columns that contain only one value: `Estimates`, `UOM`, `UOM_ID`, `SCALAR_FACTOR`, `SCALAR_ID`, and `DECIMALS`. They do not provide useful information for analysis, so we will remove them. 

In the last tutorial, you ran `df.isna().sum()` to see how many values were missing from each column. Since columns `STATUS`, `SYMBOL`, and `TERMINATED` are missing almost all values, we will remove them as well. 


<div class="alert alert-block alert-success">
In the code cell below, we created a variable <code>cols_to_drop</code> with the names of the columns to remove. Complete the code to create a variable <code>clean_df</code> that refers to a DataFrame with the columns listed in cols_to_drop removed.
</div>

In [62]:
cols_to_drop = ['STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS', 'Estimates',
                 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID']

clean_df = df.drop(columns=cols_to_drop)

# Preview the cleaned DataFrame
clean_df.head()

,REF_DATE,GEO,DGUID,Rental unit type,VECTOR,COORDINATE,VALUE
0,2024-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1150.0
1,2024-04,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1050.0
2,2024-07,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1200.0
3,2024-10,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1230.0
4,2025-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1210.0


### B: Handle Missing Values

If a column only has a small number of unique values, it can be useful for categorization (i.e., as a categorical variable). For example, according to the output of `nunique` above, the column 'Rental unit type' has only three unique values. Now we'll examine what the three values are and how many there are of each.

<div class="alert alert-block alert-success">
In the cell below, create a variable <code>unit_info</code> and assign it the value returned by calling the <code>Series</code> method <code>value_counts</code>. 
</div>

In [71]:
# Reminder: use clean_df (not df) here and in all cells that follow

unit_info = clean_df['Rental unit type'].value_counts()

unit_info

Rental unit type
1 bedroom     210
2 bedrooms    210
Room          210
Name: count, dtype: int64


<div class="alert alert-block alert-success">
We saw before that the <code>VALUE</code> column was mostly complete, but contained some missing values. We'll use the median value of all non-missing values by 'Rental unit type' to fill in the missing entries. For example, the median <code>VALUE</code> for rental unit type 'Apartment - 1 bedroom' will be used to fill in any missing <code>VALUE</code>s for entries with that type of unit. Complete the following code.
</div>

In [ ]:
# Reminder: use clean_df

# TODO: 1. calculate the median values of the 'VALUE' column by 'Rental unit type'

#Self note: Transform sets every row to have the median value so that when we do fillna it becomes alot easier for the computer
medians_by_type = clean_df.groupby('Rental unit type')['VALUE'].transform('median')


# TODO: 2. fill in the missing values in the 'VALUE' column of clean_df with that unit type's median
clean_df['VALUE'] = clean_df['VALUE'].fillna(medians_by_type)


### C: Add a Column

In the code cell below, we specify that the `REF_DATE` column should be treated as `datetime`.

In [66]:
clean_df['REF_DATE'] = pd.to_datetime(clean_df['REF_DATE'])

clean_df.head()

,REF_DATE,GEO,DGUID,Rental unit type,VECTOR,COORDINATE,VALUE
0,2024-01-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1150.0
1,2024-04-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1050.0
2,2024-07-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1200.0
3,2024-10-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1230.0
4,2025-01-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1210.0


<div class="alert alert-block alert-success">
Complete the following code to create a new column called <code>YEAR</code> by extracting the year from the datetime object in the column <code>REF_DATE</code>.
</div>

In [67]:
# Reminder: use clean_df
clean_df['YEAR'] = clean_df['REF_DATE'].dt.year

### D: Rename a Column

<div class="alert alert-block alert-success">
Rename the <code>VALUE</code> column to <code>Monthly Rent</code>.
</div>

In [68]:
# Reminder: use clean_df
clean_df = clean_df.rename(columns={"VALUE":"Monthly Rent"})
clean_df.head()

,REF_DATE,GEO,DGUID,Rental unit type,VECTOR,COORDINATE,Monthly Rent,YEAR
0,2024-01-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1150.0,2024
1,2024-04-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1050.0,2024
2,2024-07-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1200.0,2024
3,2024-10-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1230.0,2024
4,2025-01-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,Apartment - 1 bedroom,v1675424962,1.2.1,1210.0,2025


### E: Clean strings in the Rental unit type column

<div class="alert alert-block alert-success">
Some of the values in the <code>Rental unit type</code> column begin with 'Apartment - '. 

Remove 'Apartment - ' from the beginning of each item in the <code>Rental unit type</code> column that contains that text.

Use the <code>Series.str.removeprefix</code> method:<br>
https://pandas.pydata.org/docs/reference/api/pandas.Series.str.removeprefix.html
</div>

In [69]:
# Reminder: use clean_df
clean_df['Rental unit type'] = clean_df['Rental unit type'].str.removeprefix("Apartment - ")
clean_df.head()

,REF_DATE,GEO,DGUID,Rental unit type,VECTOR,COORDINATE,Monthly Rent,YEAR
0,2024-01-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,1 bedroom,v1675424962,1.2.1,1150.0,2024
1,2024-04-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,1 bedroom,v1675424962,1.2.1,1050.0,2024
2,2024-07-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,1 bedroom,v1675424962,1.2.1,1200.0,2024
3,2024-10-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,1 bedroom,v1675424962,1.2.1,1230.0,2024
4,2025-01-01,"St. John's, Census metropolitan area (CMA)",2021S0503001,1 bedroom,v1675424962,1.2.1,1210.0,2025


## Final Task: Show your TA and submit to MarkUs

If you have not already done so, make sure your TA has recorded your attendance.

Submit your completed `w04_tutorial.ipynb` file to [Markus](https://markus.teach.cs.toronto.edu/markus/courses/128) and run the tests.